## Aggregate rasters
This is the workflow for aggregating predictor rasters to the coarser resolutions used for analysis (96.5 km, 193 km, and 385.9 km).

In [69]:
library(raster)
library(rnaturalearth)
library(here)
library(tictoc)
library(tidyverse)
library(furrr)

out_path <- here("data", "climate_agg")
res_list <- list(high = 96.5, medium = 193, low = 385.9)

Create template rasters to resample predictors and filter genetic data with. 

Template resolutions:  
* 96.5 km (1 degree at 30 deg latitude) {**template_high**}
* 193 km (2 degrees) {**template_medium**}
* 385.9 km (4 degrees) {**template_low**}

In [56]:
crs <- "+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +datum=WGS84 +ellps=WGS84 +units=m +no_defs"

# read in a world shape file. Might make this able to use countries as well
world_shp <- rnaturalearth::ne_coastline(scale = "small")
  
# convert the coordinate system to behrmann
world_shp <- sp::spTransform(world_shp, crs)
  
# turn it into a raster
world_raster <- raster::raster(world_shp)


# transform world raster to high resolution
template_high <- world_raster
res(template_high) <- 96500


# transform world raster to medium resolution
template_medium <- world_raster
res(template_medium) <- 193000

# transform world raster to low resolution
template_low <- world_raster
res(template_low) <- 385900

# write to data/templates/ folder
writeRaster(template_high, filename = here("data", "templates", "template_high.tif"))
writeRaster(template_medium, filename = here("data", "templates", "template_medium.tif"))
writeRaster(template_low, filename = here("data", "templates", "template_low.tif"))

Warning message in .local(x, filename, ...):
“all cell values are NA”
Warning message in .local(x, filename, ...):
“all cell values are NA”
Warning message in .local(x, filename, ...):
“all cell values are NA”


This function follows these steps:  
* reproject raster to Behrmann equal-area projection
* resample with bilinear interpolation to the coarse resolution

This process is slow, but results in rasters with the precise desired resolution in km. Alternatives result in imprecise resolutions, where for example a 96.5 km resolution specification, the resolution is actually 96.48628 km or 95 km. 

In [58]:
# function to resample rasters to any equal area grid of the three resolutions we're considering.


# crs: define the crs you want to project to. It must be an equal area projection. The default crs is behrmann equal area. 
# x: a raster or raster stack
# km: km resolution you want (area will be km x km)
# is_categorical: if is_categorical is true, using "ngb" interpolation
resample_equal_area <- function(x, 
                                crs = "+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +datum=WGS84 +ellps=WGS84 +units=m +no_defs", 
                                km,
                                is_categorical = FALSE) {
  
  # project raster to new coordinate system (this has no values- we put values back in later)
  x_ext <- raster::projectExtent(x, crs)
  
  # make the resolution square since we're interested in an equal area projection
  raster::res(x_ext) <- raster::xres(x_ext)
  
  # add original values to our projection. We can't take shortcuts like doing this instead of doing projectExtent first because it removes cells at the extreme latitudes.
  x_repro <- raster::projectRaster(x, x_ext)
  
  if (km == 96.5) {
      end_raster = template_high
  } else if (km == 193) {
      end_raster = template_medium
  } else if (km == 385.9) {
      end_raster = template_low
  } else stop("Incorrect resolution specification. Must be either 96.5, 193, or 395.9.")
  
  # resample our raster of interest to the beginning raster  
  if (is_categorical) {
      m <- "ngb"
  } else m <- "bilinear"
  x_resampled <- raster::resample(x_repro, end_raster, method = m) 

  
  return(x_resampled)
}


In [62]:
# function to write aggregated rasters to file, using a specified prefix for the file and layer names
write_raster <- function (x, prefix) {
    names(x) <- paste0(prefix, "_", names(x))
    out_file <- file.path(out_path, paste0(names(x), ".tif"))
    writeRaster(x, filename = out_file, overwrite = TRUE)
}

## LGM climate
From [paleoclim.org](paleoclim.org)

In [47]:
lgm_path <- dir(here("data", "climate_raw", "LGM_bioclim"), pattern = "*.tif$", full.names = TRUE)
lgm_rast_list <- purrr::map(lgm_path, raster)

931.556 sec elapsed


#### 96.5 km resolution

In [63]:
tic()
lgm_agg_high <- purrr::map(lgm_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(lgm_agg_high, ~write_raster(.x, prefix = "lgm")))

#### 193 km resolution

In [ ]:
lgm_agg_medium <- purrr::map(lgm_rast_list, ~resample_equal_area(.x, km = res_list$medium))
# write rasters to file
invisible(purrr::map(lgm_agg_medium, ~write_raster(.x, prefix = "lgm")))

#### 385.9 km resolution

In [ ]:
lgm_agg_low <- purrr::map(lgm_rast_list, ~resample_equal_area(.x, km = res_list$low))
# write rasters to file
invisible(purrr::map(lgm_agg_low, ~write_raster(.x, prefix = "lgm")))

[1] 96.5